In [72]:
# !pip install torchtext
# !pip install spacy
# !python -m spacy download en_core_web_sm
# !pip uninstall torchtext==0.6.0

In [33]:
import torch
import torch.nn.functional as F
import torchtext
import time
import random
import pandas as pd
import numpy as np
import tensorflow as tf

torch.backends.cudnn.deterministic = True

In [3]:
RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)

VOCABULARY_SIZE = 20000
LEARNING_RATE = 0.005
BATCH_SIZE = 128
NUM_EPOCHS = 15
DEVICE = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

EMBEDDING_DIM = 128
HIDDEN_DIM = 256
NUM_CLASSES = 3

In [4]:
df = pd.read_csv('../Reddit Data/final_dataset_encoded_2024-03-18.csv', index_col=0)
df.head()

,preprocessed-text-removal,sentiment
0,therapist told trust issue,2
1,believe,0
2,think internet ruined know anymore,1
3,hi longest time wondering enjoy anything anymo...,1
4,masturbation guilt,1


In [38]:
texts = df['preprocessed-text-removal'].tolist()

In [61]:
train, val, test = np.split(df.sample(frac=1), [int(0.8*len(df)), int(0.9*len(df))])

c:\Users\Willis\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [49]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')

def yield_tokens():
    for sentence in texts:
        if isinstance(sentence, str):
            tokens = tokenizer(sentence)
            yield tokens

token_generator = yield_tokens()

vocab = build_vocab_from_iterator(token_generator)

29220lines [00:00, 62480.69lines/s]


In [62]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test, batch_size=64, shuffle=True)

In [68]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [28]:
class RNN(torch.nn.Module):
    
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()

        self.embedding = torch.nn.Embedding(input_dim, embedding_dim)
        self.rnn = torch.nn.LSTM(embedding_dim,
                                 hidden_dim)        
        
        self.fc = torch.nn.Linear(hidden_dim, output_dim)
        

    def forward(self, text):        
        embedded = self.embedding(text)
        output, (hidden, cell) = self.rnn(embedded)
        hidden.squeeze_(0)
        output = self.fc(hidden)
        return output

In [66]:
torch.manual_seed(RANDOM_SEED)
model = RNN(input_dim=len(vocab),
            embedding_dim=EMBEDDING_DIM,
            hidden_dim=HIDDEN_DIM,
            output_dim=NUM_CLASSES
)

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

In [1]:
import torch
import torch.nn.functional as F
import torchtext
import time
import random
import pandas as pd
import numpy as np
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader

torch.backends.cudnn.deterministic = True

RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)

VOCABULARY_SIZE = 20000
LEARNING_RATE = 0.005
BATCH_SIZE = 128
NUM_EPOCHS = 15
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

EMBEDDING_DIM = 128
HIDDEN_DIM = 256
NUM_CLASSES = 3

print(DEVICE)

df = pd.read_csv('../Reddit Data/final_dataset_encoded_2024-03-18.csv', index_col=0)
texts = df['preprocessed-text-removal'].tolist()
train, val, test = np.split(df.sample(frac=1), [int(0.8*len(df)), int(0.9*len(df))])

tokenizer = get_tokenizer('basic_english')

def yield_tokens():
    for sentence in texts:
        if isinstance(sentence, str):
            tokens = tokenizer(sentence)
            yield tokens

token_generator = yield_tokens()

vocab = build_vocab_from_iterator(token_generator)

train_dataloader = DataLoader(train, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test, batch_size=64, shuffle=True)

class RNN(torch.nn.Module):
    
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()

        self.embedding = torch.nn.Embedding(input_dim, embedding_dim)
        self.rnn = torch.nn.LSTM(embedding_dim,
                                 hidden_dim)        
        
        self.fc = torch.nn.Linear(hidden_dim, output_dim)
        

    def forward(self, text):        
        embedded = self.embedding(text)
        output, (hidden, cell) = self.rnn(embedded)
        hidden.squeeze_(0)
        output = self.fc(hidden)
        return output
    
torch.manual_seed(RANDOM_SEED)
model = RNN(input_dim=len(vocab),
            embedding_dim=EMBEDDING_DIM,
            hidden_dim=HIDDEN_DIM,
            output_dim=NUM_CLASSES
)

model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

c:\Users\Willis\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


cuda


29220lines [00:00, 65259.20lines/s]


In [2]:
def compute_accuracy(model, data_loader, device):

    with torch.no_grad():

        correct_pred, num_examples = 0, 0

        for i, (features, targets) in enumerate(data_loader):

            features = features.to(device)
            targets = targets.float().to(device)

            logits = model(features)
            _, predicted_labels = torch.max(logits, 1)

            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100

In [4]:
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, batch_data in enumerate(train_dataloader):
        
        text = batch_data.TEXT_COLUMN_NAME.to(DEVICE)
        labels = batch_data.LABEL_COLUMN_NAME.to(DEVICE)

        ### FORWARD AND BACK PROP
        logits = model(text)
        loss = F.cross_entropy(logits, labels)
        optimizer.zero_grad()
        
        loss.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_dataloader):03d} | '
                   f'Loss: {loss:.4f}')

    with torch.set_grad_enabled(False):
        print(f'training accuracy: '
              f'{compute_accuracy(model, train_dataloader, DEVICE):.2f}%'
              f'\nvalid accuracy: '
              f'{compute_accuracy(model, val_dataloader, DEVICE):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_accuracy(model, test_dataloader, DEVICE):.2f}%')

KeyError: 10132

In [8]:
import torch
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
from sklearn.model_selection import train_test_split
from collections import Counter

train_data = pd.read_csv('../Reddit Data/final_dataset_encoded_2024-03-18.csv', index_col=0)

X_train, X_valid, Y_train, Y_valid= train_test_split(train_data['preprocessed-text-removal'].tolist(),\
                                                      train_data['sentiment'].tolist(),\
                                                      test_size=0.2,\
                                                      stratify = train_data['sentiment'].tolist(),\
                                                      random_state=0)


print('Train data len:'+str(len(X_train)))
print('Class distribution'+str(Counter(Y_train)))


print('Valid data len:'+str(len(X_valid)))
print('Class distribution'+ str(Counter(Y_valid)))

# print('Test data len:'+str(len(test_data['text'].tolist())))
# print('Class distribution'+ str(Counter(test_data['label'].tolist())))


train_dat =list(zip(Y_train,X_train))
valid_dat =list(zip(Y_valid,X_valid))
# test_dat=list(zip(test_data['label'].tolist(),test_data['text'].tolist()))

Train data len:23620
Class distributionCounter({1: 11092, 2: 7443, 0: 5085})
Valid data len:5906
Class distributionCounter({1: 2773, 2: 1861, 0: 1272})


In [14]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')
train_iter = train_dat
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

TypeError: build_vocab_from_iterator() got an unexpected keyword argument 'specials'

In [12]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) 

TypeError: 'Vocab' object is not callable